In [1]:
import cv2
import mediapipe as mp
import os

def split_video_on_pose(
    input_video_path ,
    output_video_path_no_person,
    output_video_path_person,
    pose_detection_confidence=0.2,
    pose_tracking_confidence=0.2
):
    """
    Splits an input video into two parts based on pose detection:
      1. output_video_path_no_person: concatenated segments where no person is detected
      2. output_video_path_person: segment(s) where a person is detected
    """

    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(
        static_image_mode=False,
        model_complexity=1,
        enable_segmentation=False,
        min_detection_confidence=pose_detection_confidence,
        min_tracking_confidence=pose_tracking_confidence
    )

    # Initialize OpenCV video capture
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {input_video_path}")
        return

    # Get video properties
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Prepare for writing videos
    # FourCC is a 4-byte code used to specify the video codec.
    # On many systems you might use 'mp4v' or 'XVID'.
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # We will first scan the entire video to find the first and last frame
    # where a person is detected (for stable detection).
    first_person_frame = None
    last_person_frame = None

    current_frame_index = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert the BGR image to RGB for MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Pose detection
        results = pose.process(rgb_frame)
        
        # Check if landmarks are detected
        if results.pose_landmarks:
            if first_person_frame is None:
                first_person_frame = current_frame_index
            last_person_frame = current_frame_index
        
        current_frame_index += 1

    # Reset capture to re-read the video from start
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    if first_person_frame is None or last_person_frame is None:
        print("No person was detected in the video. Writing the entire video to 'no_person' output.")
        # If no frames had a person, just write the entire video to "no_person" and leave "person" empty
        out_no_person = cv2.VideoWriter(output_video_path_no_person, fourcc, fps, (width, height))
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            out_no_person.write(frame)
        
        out_no_person.release()
        cap.release()
        return
    else:
        print(f"Person first detected at frame: {first_person_frame}")
        print(f"Person last detected at frame:  {last_person_frame}")

    # Create video writers
    out_no_person = cv2.VideoWriter(output_video_path_no_person, fourcc, fps, (width, height))
    out_person = cv2.VideoWriter(output_video_path_person, fourcc, fps, (width, height))

    # We'll write all frames < first_person_frame to 'no_person'
    # We'll write all frames in [first_person_frame, last_person_frame] to 'person'
    # We'll write all frames > last_person_frame to 'no_person'
    current_frame_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        if current_frame_index < first_person_frame or current_frame_index > last_person_frame:
            out_no_person.write(frame)
        else:
            out_person.write(frame)
        
        current_frame_index += 1

    # Release resources
    cap.release()
    out_no_person.release()
    out_person.release()

    print("Splitting complete.")
    print(f"No-person video saved to: {output_video_path_no_person}")
    print(f"Person video saved to: {output_video_path_person}")


if __name__ == "__main__":
    # Example usage
    input_video = r"C:\Users\joshu\Queen's University Belfast\Michael Loughran - raw\20241122_145206_BR2.mp4"

    # Output paths (ensure these include file names and extensions)
    output_no_person = r"C:\Users\joshu\Documents\EEE\AMNIS\splitter\no_person.mp4"
    output_person = r"C:\Users\joshu\Documents\EEE\AMNIS\splitter\person.mp4"

    # Run the splitting function
    split_video_on_pose(
        input_video_path=input_video,
        output_video_path_no_person=output_no_person,
        output_video_path_person=output_person
    )



Person first detected at frame: 37
Person last detected at frame:  126
Splitting complete.
No-person video saved to: C:\Users\joshu\Documents\EEE\AMNIS\splitter\no_person.mp4
Person video saved to: C:\Users\joshu\Documents\EEE\AMNIS\splitter\person.mp4
